In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import jdatetime
import re
from ipywidgets import widgets, HBox, VBox, interactive_output
from IPython.display import display

In [2]:
# --- ensure Time is datetime ---
df = pd.read_excel('Currency_with_cpi.xlsx')

# --- helper functions to filter options based on search text ---
def update_country_options(change):
    text = change['new'].lower()
    filtered = [c for c in df['Country'].unique() if text in c.lower()]
    countries_widget.options = sorted(filtered)

def update_currency_options(change):
    text = change['new'].lower()
    filtered = [c for c in df['Currency'].unique() if text in c.lower()]
    currencies_widget.options = sorted(filtered)

# --- widgets ---
country_search = widgets.Text(
    description='🔍 Country',
    placeholder='Type to search...',
    layout=widgets.Layout(width='45%')
)
countries_widget = widgets.SelectMultiple(
    options=sorted(df['Country'].unique()),
    description='Countries',
    layout=widgets.Layout(width='45%')
)
country_search.observe(update_country_options, names='value')

currency_search = widgets.Text(
    description='🔍 Currency',
    placeholder='Type to search...',
    layout=widgets.Layout(width='45%')
)
currencies_widget = widgets.SelectMultiple(
    options=sorted(df['Currency'].unique()),
    description='Currencies',
    layout=widgets.Layout(width='45%'),
    value=['IRR: Rial_Nominal']
)
currency_search.observe(update_currency_options, names='value')

time_start_widget = widgets.Text(
    # value=str(df['Time'].min().date()),
    description='Start',
    placeholder='YYYY-MM-DD',
    layout=widgets.Layout(width='45%'),
    value = '2000'
)
time_end_widget = widgets.Text(
    # value=str(df['Time'].max().date()),
    description='End',
    placeholder='YYYY-MM-DD',
    layout=widgets.Layout(width='45%'),
    value = '2025'
)

value_type_widget = widgets.Dropdown(
    options=['Value','RER', 'CPI', 'CPI_usd'],
    value='Value',
    description='Metric',
)

# --- plotting function ---
def plot_filtered(countries, currencies, time_start, time_end, metric):
    if not countries:
        countries = df['Country']
    
    if not currencies:
        currencies = df['Currency']
    
    
    try:
        tmin = pd.to_datetime(time_start)
        tmax = pd.to_datetime(time_end)
    except:
        print("❌ Invalid date format — use YYYY-MM-DD")
        return
    
    subset = df[
        (df['Country'].isin(countries)) &
        (df['Currency'].isin(currencies)) &
        (df['Time'].between(tmin, tmax))
    ]
    
    if subset.empty:
        print("⚠️ No data for this selection.")
        return

    # Pivot so countries are columns, currency ignored in pivot because multiple currencies
    pivot_df = subset.pivot_table(index='Time', columns='Country', values=metric, aggfunc='mean')
    
    pivot_df.plot(figsize=(25,7), title=f"{metric} for selected countries & currencies")
    plt.xlabel("Time")
    plt.ylabel(metric)
    plt.grid(True)
    plt.show()

# --- layout ---
ui = VBox([
    HBox([country_search, currency_search]),
    HBox([countries_widget, currencies_widget]),
    value_type_widget,
    HBox([time_start_widget, time_end_widget]),
])

out = interactive_output(
    plot_filtered,
    {
        'countries': countries_widget,
        'currencies': currencies_widget,
        'time_start': time_start_widget,
        'time_end': time_end_widget,
        'metric': value_type_widget
    }
)

display(ui, out)


Output()